In [1]:
!pip install wfdb
!pip install ast

    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Hayden Wilde\\AppData\\Local\\Temp\\pip-install-sxqn9uf4\\ast_94226de16281453dad2ceb36b075588a\\setup.py'"'"'; __file__='"'"'C:\\Users\\Hayden Wilde\\AppData\\Local\\Temp\\pip-install-sxqn9uf4\\ast_94226de16281453dad2ceb36b075588a\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Hayden Wilde\AppData\Local\Temp\pip-pip-egg-info-mpwudusq'
         cwd: C:\Users\Hayden Wilde\AppData\Local\Temp\pip-install-sxqn9uf4\ast_94226de16281453dad2ceb36b075588a\
    Complete output (7 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>


In [ ]:
##SAMPLE CODE FROM DATASET INFO PAGE - IMPORTS DATA AND SPLITS INTO TEST/TRAIN SET

import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate, path):
    #choose 100 for the 100hz file (500mb) and 500hz (2.7gB)
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = r"C:\Users\Hayden Wilde\PYTHON\ML Project\PTB-XL Extracted\\"
sampling_rate = 100


# load and convert annotation data
Y = pd.read_csv(r"C:\Users\Hayden Wilde\PYTHON\ML Project\PTB-XL Extracted\ptbxl_database.csv", index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass